In [28]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm
from tsfresh import extract_features
from lightgbm import LGBMRegressor

In [2]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [3]:
# Загрузка данных
# windows
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

In [4]:
# from sklearn.model_selection import TimeSeriesSplit

# N_FOLDS = 5
# ts_splitter = TimeSeriesSplit(N_FOLDS)

### Важно! В train.csv и test.csv признаки `epoch` и `sat_id` стоят в разном порядке. Это может быть критично для некоторых моделей (например, lgbm). Поэтому сделаем единный порядок:

In [15]:
test_cols = list(full_test.columns)
test_cols[0] = 'epoch'
test_cols[1] = 'sat_id'
full_test = full_test.reindex(columns=test_cols)

In [16]:
full_train['epoch'] = full_train.epoch.apply(lambda x: time.mktime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').timetuple())).values
full_test['epoch'] = full_test.epoch.apply(lambda x: time.mktime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').timetuple())).values

In [17]:
full_train.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,,
0,1.388520e+09,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1.388523e+09,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,1.388526e+09,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,1.388528e+09,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,1.388531e+09,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [9]:

# kind_to_fc_parameters = {
#     "x_sim": {"mean": None},#, "max": None, "min": None},
#     "y_sim": {"mean": None},#, "max": None, "min": None},
#     "z_sim": {"mean": None},#, "max": None, "min": None},
#     "Vx_sim": {"mean": None},#, "max": None, "min": None},
#     "Vy_sim": {"mean": None},#, "max": None, "min": None},
#     "Vz_sim": {"mean": None}#, "max": None, "min": None},
    
}

# extracted_features = extract_features(full_train, kind_to_fc_parameters=None,
#                                       column_id="sat_id", column_sort="epoch")

In [18]:
# import pickle

In [19]:
# with open('timeseries_features', 'wb') as handle:
#     pickle.dump(extracted_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# extracted_features

In [21]:
full_train.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,,
0,1.388520e+09,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1.388523e+09,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,1.388526e+09,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,1.388528e+09,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,1.388531e+09,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [22]:
# x_num

for i, col in enumerate(['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']):
    full_train[col + '_num'] = 0
    arr = full_train[col + '_num'].values
    start = 0
    for sat_id in range(600):
        df = full_train[full_train.sat_id == sat_id]
        step = df.shape[0]
        idx_min = df[col].idxmin() % 24
        arr[start: start + step] = np.fromfunction(lambda i: (i + idx_min) % 24, (step, ))
        start += step
    full_train[col + '_num'] = arr

In [23]:
# x_num

for i, col in enumerate(['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']):
    full_test[col + '_num'] = 0
    arr = full_test[col + '_num'].values
    start = 0
    for sat_id in full_test.sat_id.unique():
        df = full_test[full_test.sat_id == sat_id]
        step = df.shape[0]
        idx_last = full_train[full_train.sat_id == sat_id][col + '_num'].iloc[-1] + 1
        arr[start: start + step] = np.fromfunction(lambda i: (i + idx_last) % 24, (step, ))
        start += step
    full_test[col + '_num'] = arr
        

In [24]:
for col_name in ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']:
    full_test[col_name] = 0.0

In [ ]:
%%time

features_for_tsfresh = ['sat_id', 'epoch', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
target_cols = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

width = 8
for sat_id in tqdm(full_test.sat_id.unique()):
    df1_train = full_train[full_train.sat_id == sat_id]
    df1_test =  full_test[full_test.sat_id == sat_id]
    for col in target_cols:
        for i in range(24):
            df2_train = df1_train[df1_train[col + '_sim_num'] == i]
            df2_test = df1_test[df1_test[col + '_sim_num'] == i]
            
            train_features = extract_features(df2_train[features_for_tsfresh], kind_to_fc_parameters=None,
                                              column_id="sat_id", column_sort="epoch")
            test_features = extract_features(df2_test[features_for_tsfresh], kind_to_fc_parameters=None,
                                              column_id="sat_id", column_sort="epoch")
            tmp = np.zeros((df2_train.shape[0], train_features.size))
            tmp = train_features.values + tmp
            df2_train = pd.concat([df2_train, pd.DataFrame(tmp, index=df2_train.index)], axis=1)
            
            tmp = np.zeros((df2_test.shape[0], test_features.size))
            tmp = test_features.values + tmp
            df2_test = pd.concat([df2_test, pd.DataFrame(tmp, index=df2_test.index)], axis=1)
            
            
            lin_model = LGBMRegressor()
            X_train = df2_train.reset_index().drop(columns=target_cols).values[-width:]
            y_train = df2_train[col].values[-width:]
            lin_model.fit(X_train, y_train)
            X_test = df2_test.reset_index().drop(columns=target_cols).values[-width:]
            y_test = lin_model.predict(X_test)
            full_test.loc[X_test[:, 0].ravel(), col] = y_test

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 69.86it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 69.30it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 85.29it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 79.01it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 79.28it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 83.94it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 84.63it/s]

In [ ]:
full_test[['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('submission.csv', index_label='id')